In [1]:
import ee
import geemap.eefolium as geemap
import os
import requests
import math
import json

Map = geemap.Map()
Map

In [2]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [3]:
#MASKING FUNCTION FOR L5 AND L7
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

In [12]:
def generateLandsatImage(year):
    if(year<1999):
        dataset_land = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    elif(year<2013):
        dataset_land = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(cloudMaskL457) \

        image_land = dataset_land.select(['B3', 'B2', 'B1']).median()
        land_vis['bands'] = ['B3', 'B2', 'B1']
    else:
        dataset_land = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31') \
                      .map(maskL8sr) \

        image_land = dataset_land.select(['B4', 'B3', 'B2']).median()
        land_vis['bands'] = ["B4","B3","B2"]

    url_land = image_land.getThumbURL(land_vis)
    return url_land

In [5]:
def generateWaterMapImage(year):
    dataset_water = ee.ImageCollection("JRC/GSW1_2/YearlyHistory") \
                      .filterDate(str(year)+'-01-01', str(year)+'-12-31')
    image_water = dataset_water.select(['waterClass']).median()
    url_water = image_water.getThumbURL(water_vis)
    return url_water

In [6]:
def exportLocally(URL, fileName):
    r = requests.get(URL, stream=True)
    out_png = os.path.expanduser('~/Desktop/picture_samples/'+fileName+'.png')

    if r.status_code != 200:
        print('An error occurred while downloading.')
    else:
        with open(out_png, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)
        print(fileName+ ' created!')

In [8]:
def generatePhotos(lat, lng, startY, endY):
    for year in range(startY,endY,1):
        fileName = str(lat) +'_'+ str(lng) +'_'+ str(year) +'_'
        landURL = generateLandsatImage(year)
        waterURL = generateWaterMapImage(year)
        exportLocally(landURL, fileName+'land')
        exportLocally(waterURL, fileName+'water')

In [9]:
##CHANGE FOR NUMBER OF ITERATIONS
START_YEAR = 2018 # MAX = 1985, MIN = 2019 ----- MAX 1984 IS POSSIBLE BUT DATA USUALLY NOT THAT GOOD
END_YEAR = 2020 # should be 2020 always, change only for testing

In [18]:
with open('../ReactJs/src/test.json') as json_file:
    data = json.load(json_file)
    for coords in data:
        
        #THIS SHOULD BE LATITUDE, LONGITUDE, LATITUDE, LONGITUDE
        #BUT FOR SOME BLOODY REASON ee TAKES THE REVERSE ????
        roi = ee.Geometry.Rectangle([coords['west'], coords['north'], coords['east'], coords['south']])

        #IMAGE VISUALIZATION PARAMETERS
        land_vis = {
            'gamma': 1.4,
            'max': 3000,
            'min': 0,
            'scale': 10,
            'region': roi,
            'format': 'png'
            }

        water_vis = {
            'bands': ['waterClass'],
            'palette': ["cccccc","ffffff","99d9ea","0000ff"],
            'max': 3,
            'min': 0,
            'scale': 10,
            'region': roi,
            'format': 'png'
        }
    
        generatePhotos(round(coords['latitude'],4), round(coords['longitude'],4), START_YEAR, END_YEAR)

-3.7957_-38.9734_2018_land created!
-3.7957_-38.9734_2018_water created!
-3.7957_-38.9734_2019_land created!
-3.7957_-38.9734_2019_water created!
-3.9245_-38.5656_2018_land created!
-3.9245_-38.5656_2018_water created!
-3.9245_-38.5656_2019_land created!
-3.9245_-38.5656_2019_water created!
-4.0725_-38.5477_2018_land created!
-4.0725_-38.5477_2018_water created!
-4.0725_-38.5477_2019_land created!
-4.0725_-38.5477_2019_water created!
-4.2341_-38.4186_2018_land created!
-4.2341_-38.4186_2018_water created!
-4.2341_-38.4186_2019_land created!
-4.2341_-38.4186_2019_water created!
-4.0875_-39.4555_2018_land created!
-4.0875_-39.4555_2018_water created!
-4.0875_-39.4555_2019_land created!
-4.0875_-39.4555_2019_water created!
-3.7697_-39.3786_2018_land created!
-3.7697_-39.3786_2018_water created!
-3.7697_-39.3786_2019_land created!
-3.7697_-39.3786_2019_water created!
